<a href="https://colab.research.google.com/github/scaomath/wustl-math450/blob/main/Math_450_Notebook_2_(From_Numpy_to_PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding Lecture 2 of Math 450

Goal of the our class: make us learn machine learning in torch package.
- Build our own neural net using Torch's LEGO-like blocks.
- Write torch-like code from scratch.
- Write our own optimizer.


This is a worksheet version of the notebook. We can follow along during the coding lecture and then download the annotated version in our Github repository.

Download this notebook at: https://github.com/scaomath/wustl-math450/blob/main/Math_450_Notebook_2_(From_Numpy_to_PyTorch).ipynb

In [ ]:
import numpy as np
import torch

In [ ]:
# x = np.array([0, 1, 2, 3])
x = torch.tensor([0, 1, 2, 3], dtype=torch.uint8)
print(x)

tensor([0, 1, 2, 3], dtype=torch.uint8)


In [ ]:
# np.all(x)
torch.all(x) # ctrl/cmd + enter to run this cell
# 0 output here just means the output is False

tensor(0, dtype=torch.uint8)

In [ ]:
x.all()

tensor(0, dtype=torch.uint8)

## Review of Coding Lecture 1: From Numpy to Torch

- Matrix vector multiplication (`dot` in `numpy` and `mm` in `torch`) vs `*` (elementwise multiplication)
- Axes of an array, `squeeze()`.
- Object-oriented way of applying functions.
- `reshape()` in `numpy` vs `view()` in torch.

In [ ]:
# matrix vector multiplication vs *
x = np.array([[1,2], [0,5]])
y = np.array([1.3, 2.5])

In [ ]:
print(x, '\n\n', y)

[[1 2]
 [0 5]] 

 [1.3 2.5]


In [ ]:
y.shape

(2,)

In [ ]:
x*y # this is not using the matrix x multiplied by y
# [1.3 2.5] is multiplied on each row of x

array([[ 1.3,  5. ],
       [ 0. , 12.5]])

In [ ]:
y.reshape(-1,1) # adding a second dimension
# change this (2,) vector to a (2,1) vector

array([[1.3],
       [2.5]])

In [ ]:
x*y.reshape(-1,1)

array([[ 1.3,  2.6],
       [ 0. , 12.5]])

In [ ]:
# matrix vector multiplication
# np.matmul
# np.dot (inner product)
x = np.arange(10).reshape(2,-1) # (2,5)
y = np.random.randn(5)
print(x, '\n\n', y)
z = x.dot(y) # (2,n) dot with (n,) yield a (2, )
# toggle comment/uncomment shortcut is ctrl/cmd+ /
print('\n', z)

[[0 1 2 3 4]
 [5 6 7 8 9]] 

 [ 0.82762389  0.66481707  0.03514183  0.18600171 -0.13526512]

 [0.75204542 8.64364239]


In [ ]:
a = np.random.randn(5, 3) # (5,3)
b = np.random.randn(3, 100)
print(a.dot(b).shape)

(5, 100)


In [ ]:
a = torch.randn( (5,3) )
b = torch.randn( (3,100) )
print(a.mm(b).size())

torch.Size([5, 100])


In [ ]:
x=torch.arange(10).reshape(2,5)
print(x)
# axis 0 is row
# axis 1 is column

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])


In [ ]:
x.sum(dim=0) # whenever a function is applied along an axis
# that axis is gone
# (2,5) array sum along axis 0, only (5,) is left

tensor([ 5,  7,  9, 11, 13])

In [ ]:
x.sum(dim=1)

tensor([10, 35])

In [ ]:
x = torch.tensor([[1,2,3,0]])
print(x.size()) # x has a singleton dimension

torch.Size([1, 4])


In [ ]:
print(x)
print(x.size())
print(x.squeeze())
print(x.squeeze().size())

tensor([[1, 2, 3, 0]])
torch.Size([1, 4])
tensor([1, 2, 3, 0])
torch.Size([4])


In [ ]:
arr = np.arange(20).reshape(-1,5)
print(arr)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


In [ ]:
arr = torch.arange(20).view(-1,5) # same as reshape
print(arr)

tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])


# Key components for a neural network


## Multi-layer, multiple perceptrons per layer
If we have $m$ perceptrons in a single layer, for example layer 2:
<img src="https://sites.wustl.edu/scao/files/2021/02/neural_net_3-layer.png" alt="drawing" width="800"/>

Our neural network has parameters $(W, b) := \big(W^{(1)},b^{(1)},W^{(2)},b^{(2)}\big)$.

* $W^{(l)} = \big(w^{(l)}_{ij}\big)$ to denote the weight matrix, where the entry-$ij$ is associated with the connection between unit $j$ in layer $l$, and unit $i$ in layer $l+1$. Note the order of the indices, $j$ is the closer to the input that this matrix is acting on 

* $b^{(l)}_i$ is the bias associated with unit $i$ in layer $l+1$. 

In our example above, we have $W^{(1)}\in \mathbb{R}^{3×2}$, and $W^{(2)}\in \mathbb{R}^{1×3}$. Note that bias units do not have inputs or connections going into them, we write their output the value $+1$ for convenience. When we count the number of units in layer $l$, we do not count the bias unit.



## Matrix-vector representation
If we allow the activation function $f(\cdot)$ to act on vectors in an element-wise fashion: $f([\mathbf{z}_1,\mathbf{z}_2,\mathbf{z}_3])=[f(\mathbf{z}_1),f(\mathbf{z}_3),f(\mathbf{z}_3)]$, then we can write the equations above more compactly as:
$$\begin{aligned}
\mathbf{z}^{(2)} &= W^{(1)} \mathbf{x} + \mathbf{b}^{(1)} \\
\mathbf{a}^{(2)} &= f(\mathbf{z}^{(2)}) \\
\mathbf{z}^{(3)} &= W^{(2)} \mathbf{a}^{(2)} + \mathbf{b}^{(2)} \\
h(\mathbf{x}; W, b) &= \mathbf{a}^{(3)} = \mathbf{z}^{(3)}
\end{aligned}
$$
More generally, if we have an arbitrary number of layers, recalling that $\mathbf{a}^{(0)}=\mathbf{x}$ also denotes the values from the input layer, then given layer $l$'s activations $\mathbf{a}^{(l)}$, we can compute layer $(l+1)$'s activations $\mathbf{a}^{(l+1)}$ as:
$$
\begin{aligned}
\mathbf{z}^{(l+1)} &= W^{(l)} \mathbf{a}^{(l)} + \mathbf{b}^{(l)}   \\
\mathbf{a}^{(l+1)} &= f(\mathbf{z}^{(l+1)}),
\end{aligned}
$$
except the last layer where we do not need any activation.
By organizing the parameters in matrices and using matrix-vector operations, we can take advantage of fast linear algebra routines to quickly perform calculations in our network.

In [ ]:
# generate all the x, z, a variables above
x = torch.tensor([0., 1.]).view(-1, 1) 
# add a singleton dimension to a vector
print(x)
W1 = torch.randn( (3,2) )
b1 = torch.randn( (3,1) )
print(W1)
z2 = W1.mm(x) + b1 # Wx+b
print(z2)

tensor([[0.],
        [1.]])
tensor([[ 0.9907, -0.2160],
        [ 0.1002, -2.2680],
        [ 2.4253,  0.3178]])
tensor([[-0.1301],
        [-1.3717],
        [ 0.9358]])


In [ ]:
# activation
a2 = z2[z2>0]
print(a2) # returns only the positive number
a2 = z2.clamp(min=0)
print(a2)

tensor([0.9358])
tensor([[0.0000],
        [0.0000],
        [0.9358]])


In [ ]:
# from input to first layer (hidden layer)

In [ ]:
# from the first layer to the output layer

## Actual data in batch

In the actual implementation, the data normaly comes in batch, i.e., a matrix. For example, input is a matrix $X \in \mathbb{R}^{N \times d}$, $N$ is a number of samples in a batch, each row represents a sample $\mathbf{x} \in \mathbb{R}^{1\times d}$. The weight matrix $W$ is actually formulated as:
$$
W = \left(
\begin{array}{cccc}| & | & | & | \\
\mathbf{w}_1 & \mathbf{w}_2 & \cdots & \mathbf{w}_m \\
| & | & | & |
\end{array}\right),
$$
if the output dimension of the layer of interest is $m$. The vectorized formulation is, for example, from the input (layer 0, dimension $d$) to layer 1 (dimension $m$)
$$
A^{(1)} = X (W^{(0)})^{\top} + B
$$
where $X \in \mathbb{R}^{N \times d}$, $W^{(0)} \in \mathbb{R}^{m\times d}$ (input from $d$ perceptrons, output from $m$ perceptrons), $B$ is a matrix with each row being the same $\mathbf{b} \in \mathbb{R}^{1\times m}$ (layer 1 has $m$ perceptrons and has $m$ biases if applicable).

## Torch's nn module

We will demo this batch-based operation using `torch`'s neural network module `nn`. `nn.Linear` applies an (affine) linear transformation to the incoming data:
$$
Y = X W^{\top} + \mathbf{b}
$$

Reference: https://pytorch.org/docs/stable/nn.html

In [ ]:
import torch.nn as nn

In [ ]:
# Linear layer example
layer = nn.Linear(4, 3) # Wx+b transforms (4,1) vector to (3,1) vector
input = torch.randn(32, 4) # 32 samples, each sample has 4 features
print(input)

tensor([[-1.1355, -2.4036,  0.7420,  1.1788],
        [-0.9597, -1.2805,  0.3954,  1.0836],
        [-0.5114, -1.2134,  0.4472, -0.2437],
        [ 0.0647,  0.7331,  1.4567, -1.6784],
        [ 0.2293, -0.3503,  0.6469,  0.4768],
        [-0.0895, -1.3557, -0.9072, -0.3229],
        [ 0.8917,  0.0797,  0.2411, -2.2043],
        [-0.1207,  1.1065,  0.9592,  0.1088],
        [-0.3837, -0.0036, -1.0944,  0.5760],
        [-0.8434,  0.6975,  0.2265,  1.6542],
        [ 0.3379,  0.3547, -1.9871, -2.1632],
        [ 1.9732,  0.1704,  0.2612,  1.1806],
        [-0.8772,  0.9680,  1.6261,  1.0237],
        [ 0.4438, -0.8438, -0.6925, -0.0930],
        [-0.1747,  1.8252, -1.0535,  0.0892],
        [ 0.9088, -1.7673, -0.0186,  0.8402],
        [-1.2256,  1.8405, -0.3412, -0.6917],
        [-1.6789,  2.0006, -0.0667,  0.8853],
        [ 1.0742,  0.8501, -1.1665, -1.1089],
        [ 1.1039, -0.5737,  1.8888, -2.7325],
        [-0.2516, -1.3357,  0.0212,  1.1498],
        [-0.1276,  1.7405, -0.7649

In [ ]:
output = layer(input)
print(output.size())
print(output)

torch.Size([32, 3])
tensor([[-1.6478,  1.5332, -0.6837],
        [-1.1449,  1.1176, -0.2567],
        [-0.8056,  1.0426, -0.2656],
        [ 0.4978,  0.3489, -0.2216],
        [-0.1824,  0.4108, -0.5308],
        [-0.8561,  0.9125,  0.1625],
        [ 0.4602,  0.2597, -0.1179],
        [ 0.3510,  0.1334, -0.0532],
        [-0.5216,  0.5214,  0.6412],
        [-0.3496,  0.3978,  0.2650],
        [ 0.0590,  0.3697,  1.2461],
        [ 0.6387, -0.5195, -1.0481],
        [-0.0160,  0.3999, -0.2131],
        [-0.4177,  0.5197, -0.0526],
        [ 0.3726, -0.0873,  1.1313],
        [-0.6082,  0.5409, -0.9713],
        [ 0.1077,  0.4088,  1.3626],
        [-0.1206,  0.3945,  1.2499],
        [ 0.5765, -0.1708,  0.5355],
        [ 0.5341,  0.4447, -1.1011],
        [-0.9299,  0.8416, -0.4100],
        [ 0.4303, -0.0387,  1.0079],
        [-0.2079,  0.4867,  0.8137],
        [ 0.0960,  0.4268, -0.6392],
        [-0.4921,  0.4753, -0.0060],
        [ 0.7064, -0.0803, -0.8695],
        [-0.6128, 

# MNIST

"MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike."

[Read more.](https://www.kaggle.com/c/digit-recognizer)


<a title="By Josef Steppan [CC BY-SA 4.0 (https://creativecommons.org/licenses/by-sa/4.0)], from Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:MnistExamples.png"><img width="512" alt="MnistExamples" src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png"/></a>

This code is adopted from the pytorch examples repository. 
It is licensed under BSD 3-Clause "New" or "Revised" License.
Source: https://github.com/pytorch/examples/
LICENSE: https://github.com/pytorch/examples/blob/master/LICENSE


In [ ]:
from __future__ import print_function
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import matplotlib.pyplot as plt


In [ ]:
train = datasets.MNIST('../data', train=True, download=True, transform = transforms.ToTensor())

In [ ]:
train_loader = DataLoader(train, batch_size=1, shuffle=True, num_workers=2,)

In [ ]:
data_iter = iter(train_loader)
images, labels = next(data_iter)

In [ ]:
im = make_grid(images)
plt.imshow(np.transpose(im.numpy(), (1, 2, 0)))